# Data Cleaning and Preprocessing Notebook

From our earlier analysis in the EDA notebook, we've identified that some features in our dataset have missing values.

Let's start by checking the percentage of missing values in each feature.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

train_df = pd.read_csv('../data_details/train.csv')
test_df = pd.read_csv('../data_details/test.csv')

In [ ]:
# Calculating the percentage of missing values in each feature
missing_values = train_df.isnull().mean() * 100

# Displaying features with their corresponding percentage of missing values
missing_values[missing_values > 0].sort_values(ascending=False)


Features like PoolQC, MiscFeature, Alley, Fence, and FireplaceQu are missing more than 45% of their values.

Based on the high percentage of missing values we can assume that the absence of a value might indicate the absence of the feature itself. For instance, a NaN (or missing value) in PoolQC likely means that the house doesn't have a pool. Similarly, a NaN in GarageType likely indicates that there is no garage.

So, I choose to handle these cases by replacing the missing values with 'None' or 0, indicating that the house lacks these features.

For features like LotFrontage, which represents the linear feet of street connected to the property, we can use imputation. 

In real estate and the housing industry,  in a city, lot frontage is likely to be similar for houses in the same neighborhood.

So I decide to impute missing values in LotFrontage with the median LotFrontage of the neighborhood. 

The feature Electrical has just one missing value, and so we can replace it with the mode of the column without introducing much bias.



In [ ]:
# Filling missing values for features where NaN means the feature doesn't exist
for feature in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtCond', 'BsmtQual',
                'MasVnrType']:
    train_df[feature].fillna('None', inplace=True)

# Filling missing values for features where NaN means zero
for feature in ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 
                'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 
                'BsmtHalfBath', 'MasVnrArea']:
    train_df[feature].fillna(0, inplace=True)
    
# Group by neighborhood and fill in missing LotFrontage value by the median LotFrontage of the neighborhood
train_df['LotFrontage'] = train_df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# For Electrical, we'll replace it with the most common value
train_df['Electrical'].fillna(train_df['Electrical'].mode()[0], inplace=True)

# Checking if there are any missing values left
train_df.isnull().sum().sum()


### Remove missing values from test set as well

I'll do the same thing.

In [ ]:
# Calculating the percentage of missing values in each feature
missing_values = test_df.isnull().mean() * 100

# Displaying features with their corresponding percentage of missing values
missing_values[missing_values > 0].sort_values(ascending=False)


In [ ]:
# Filling missing values for features where NaN means the feature doesn't exist
for feature in ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu',
                'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
                'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'BsmtCond', 'BsmtQual',
                'MasVnrType']:
    test_df[feature].fillna('None', inplace=True)

# Filling missing values for features where NaN means zero
for feature in ['GarageYrBlt', 'GarageArea', 'GarageCars', 'BsmtFinSF1', 
                'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 
                'BsmtHalfBath', 'MasVnrArea']:
    test_df[feature].fillna(0, inplace=True)
    
# Group by neighborhood and fill in missing LotFrontage value by the median LotFrontage of the neighborhood
test_df['LotFrontage'] = test_df.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# For Electrical, we'll replace it with the most common value
test_df['Electrical'].fillna(test_df['Electrical'].mode()[0], inplace=True)

# Checking if there are any missing values left
test_df.isnull().sum().sum()


In [ ]:
test_df.isnull().sum()

In [ ]:
# I'll just fill in with the mode

mode_value = test_df['MSZoning'].mode()[0]
test_df['MSZoning'].fillna(mode_value, inplace=True)

mode_value = test_df['SaleType'].mode()[0]
test_df['SaleType'].fillna(mode_value, inplace=True)

In [ ]:
test_df.isnull().sum()

In [ ]:
# next, identify categorical values to encode

# Identifying categorical features
categorical_features = train_df.select_dtypes(include=['object']).columns

# Displaying the categorical features
categorical_features


Encoding all these features will significantly increase our dataset's dimensionality, and could make our models more complex and harder to interpret.

I have a feeling that not all of these features will have a significant impact on the SalePrice. For example, Street (the type of road access to the property) may not be as important as the entire Neighborhood in determining the price of a house.

So I consider using a feature selection method to identify the most important features, and then can use only these for one-hot encoding.

One common method to do this is to calculate the ANOVA F-value between each categorical variable and the target variable SalePrice which measures the impact of each categorical feature on the variance of SalePrice. Features with a higher F-value have a bigger impact.



In [ ]:
from sklearn.feature_selection import f_classif

# Creating an empty dataframe to store the F-values and p-values
f_values = pd.DataFrame()

# Looping through each categorical feature
for feature in categorical_features:
    # Using label encoding to convert categories into numbers
    train_df[feature] = train_df[feature].astype('category').cat.codes

    # Calculating the F-value and p-value
    f_val, p_val = f_classif(train_df[[feature]], train_df['SalePrice'])

    # Storing the F-value and p-value in the dataframe
    f_values.at[feature, 'F-value'] = f_val[0]
    f_values.at[feature, 'p-value'] = p_val[0]

# Sorting the features by F-value in descending order
f_values.sort_values('F-value', ascending=False, inplace=True)

# Displaying the F-values and p-values
f_values


The F-values give us an idea of how much each feature contributes to the variance in SalePrice. A higher F-value indicates a larger contribution. Features ExterQual, KitchenQual, BsmtQual, Street, GarageFinish, and MSZoning have high F-values, which show that they might be significant predictors for our models.

While features like Utilities, Condition1, Functional, and SaleType have very low F-values, which might mean they have less predictive power.

I choose to select the top 10 categorical features based on their F-values for one-hot encoding. 10 is just random.

In [ ]:
# Select the top 10 categorical features
selected_features = f_values.index[:10]

# One-hot encode the selected features
one_hot_df = pd.get_dummies(train_df[selected_features], drop_first=True)

# Create a new DataFrame with the one-hot encoded features and target variable
encoded_train_df = pd.concat([train_df[['SalePrice']], one_hot_df], axis=1)

# Display the first few rows of the new DataFrame
encoded_train_df.head()

Feature engineering

In [ ]:
# Create new features based on EDA and domain knowledge

# Total Area
area_features = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF']
train_df['TotalArea'] = train_df[area_features].sum(axis=1)
test_df['TotalArea'] = test_df[area_features].sum(axis=1)

# Age of House at Sale
train_df['AgeAtSale'] = train_df['YrSold'] - train_df['YearBuilt']
test_df['AgeAtSale'] = test_df['YrSold'] - test_df['YearBuilt']

# Total Bathrooms
bath_features = ['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath']
train_df['TotalBath'] = train_df['FullBath'] + 0.5*train_df['HalfBath'] + train_df['BsmtFullBath'] + 0.5*train_df['BsmtHalfBath']
test_df['TotalBath'] = test_df['FullBath'] + 0.5*test_df['HalfBath'] + test_df['BsmtFullBath'] + 0.5*test_df['BsmtHalfBath']

# Total Porch Area
porch_features = ['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
train_df['TotalPorchSF'] = train_df[porch_features].sum(axis=1)
test_df['TotalPorchSF'] = test_df[porch_features].sum(axis=1)

# Presence of Pool
train_df['HasPool'] = train_df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)
test_df['HasPool'] = test_df['PoolArea'].apply(lambda x: 1 if x > 0 else 0)

# Remodeling Indicator
train_df['Remodeled'] = (train_df['YearBuilt'] != train_df['YearRemodAdd']).astype(int)
test_df['Remodeled'] = (test_df['YearBuilt'] != test_df['YearRemodAdd']).astype(int)

# Display the first few rows of the training data
train_df.head()
